In [1]:
DATA_NAME = 'coco-outdoor' 
TRANSFORM = 'wavelet-horizontal'
CHANNEL = 'red'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_red_horizontal_wavelet_outdoor_coco.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
    cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9
2,7638.15,-0.02,-0.01,-0.03,-0.01,-0.00,-0.00,0.00
3,-0.02,1626.73,-0.00,0.01,0.00,0.00,0.00,-0.00
4,-0.01,-0.00,341.01,-0.01,-0.00,0.00,-0.00,0.00
5,-0.03,0.01,-0.01,65.81,0.00,-0.00,0.00,-0.00
6,-0.01,0.00,-0.00,0.00,12.94,-0.00,0.00,0.00
7,-0.00,0.00,0.00,-0.00,-0.00,2.52,-0.00,-0.00
8,-0.00,0.00,-0.00,0.00,0.00,-0.00,0.54,0.00
9,0.00,-0.00,0.00,-0.00,0.00,-0.00,0.00,0.14


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8,9
2,1.00000,-0.00001,-0.00000,-0.00004,-0.00004,-0.00001,-0.00004,0.00002
3,-0.00001,1.00000,-0.00000,0.00002,0.00001,0.00003,0.00000,-0.00003
4,-0.00000,-0.00000,1.00000,-0.00004,-0.00001,0.00006,-0.00003,0.00003
5,-0.00004,0.00002,-0.00004,1.00000,0.00002,-0.00000,0.00001,-0.00004
6,-0.00004,0.00001,-0.00001,0.00002,1.00000,-0.00001,0.00003,0.00006
7,-0.00001,0.00003,0.00006,-0.00000,-0.00001,1.00000,-0.00001,-0.00002
8,-0.00004,0.00000,-0.00003,0.00001,0.00003,-0.00001,1.00000,0.00002
9,0.00002,-0.00003,0.00003,-0.00004,0.00006,-0.00002,0.00002,1.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

0.056158122551362426

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[7.30005603e+06 3.24151400e+05 1.38481107e+04 4.95489283e+02
 1.79666790e+01 6.09901149e-01 2.21738563e-02 2.96027968e-33]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7
0,0.999473,-0.031815,-0.006338,-0.001224,-0.000242,-0.000047,-0.000010,-0.000003
1,0.031552,0.998805,-0.036689,-0.006725,-0.001319,-0.000256,-0.000055,-0.000015
2,0.007432,0.036160,0.998476,-0.040293,-0.007578,-0.001468,-0.000318,-0.000083
3,0.001716,0.008047,0.039559,0.997821,-0.051247,-0.009521,-0.002041,-0.000542
4,0.000421,0.001967,0.009405,0.050105,0.996306,-0.067583,-0.013811,-0.003640
5,0.000107,0.000502,0.002396,0.012451,0.064939,0.991180,-0.111456,-0.027592
6,0.000032,0.000150,0.000718,0.003714,0.018915,0.099463,0.960314,-0.259868
7,0.000018,0.000085,0.000405,0.002102,0.010676,0.054851,0.255302,0.965243


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[0.0005270899687871422,
 0.001195041148326914,
 0.0015241888912388424,
 0.0021793948515934636,
 0.0036940392903785924,
 0.008819795581430978,
 0.03968577672428708,
 0.03475727368471648]